# DATA CLEANING -PART (1)

> Data cleaning is the most challenging part of any data science project and especially when you create you own data for analysis. It is more of creating a consistent data for analysis.

In [1]:
# libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re

warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

#### Loading the Dataset

In [2]:
jobs = pd.read_csv('job_postings.csv')

#### Removing the duplicates and keep the first record.

In [3]:
jobs.drop_duplicates(subset=['job_area', 'job_classification', 'job_company','job_description',
                             'job_location', 'job_salary','job_sub_classification', 'job_title', 
                             'job_type'], keep='first', inplace= True)

#### Looking at the shape and head of the data.

In [4]:
jobs.shape

(1247, 10)

In [5]:
jobs.head(2)

,Unnamed: 0,job_area,job_classification,job_company,job_description,job_location,job_salary,job_sub_classification,job_title,job_type
0,0,CBD & Inner Suburbs,Science & Technology,SEEK Limited,The Opportunity We're looking for an experienc...,Melbourne,Base + Super + Profit + Awesome Problems to So...,"Mathematics, Statistics & Information Sciences",Senior Data Scientist,Full Time
1,1,Parramatta & Western Suburbs,Information & Communication Technology,ecareer employment services,Put Data to best useDesign & Develop BI Soluti...,Sydney,$130-140K Package,Developers/Programmers,Senior Data Engineer / BI Developer - MS BI Su...,Full Time


I am getting a Column **"Unnamed: 0"** which I need to remove, and will also get rid of **"job_sub_classification"** as I already have the broader Classification to what the **job_title** belong.

#### Dropping unnecessary columns

In [6]:
jobs= jobs.drop(['Unnamed: 0','job_sub_classification'],axis=1)

#### Resetting the index 

In [7]:
jobs.reset_index(inplace=True,drop=True)

> I used regular expression to clean various columns as per the need.

### Job Area

In [8]:
def change1(text):
    text = text.replace('&','and')
    text = text.replace(',','')
    text = text.replace('.','')
    text = text.replace('$','')
    return text

In [9]:
jobs['job_area']=jobs['job_area'].apply(lambda x: change1(x))

In [10]:
jobs["job_area"] = jobs["job_area"].str.replace('South West and M5 Corridor','South Western Suburbs')

In [11]:
jobs["job_area"].head(2)

0             CBD and Inner Suburbs
1    Parramatta and Western Suburbs
Name: job_area, dtype: object

### Job Classification

In [12]:
jobs['job_classification']=jobs['job_classification'].apply(lambda x: change1(x))

In [13]:
jobs["job_classification"].head(2)

0                      Science and Technology
1    Information and Communication Technology
Name: job_classification, dtype: object

### Job Company

In [14]:
jobs['job_company']=jobs['job_company'].apply(lambda x: change1(x))

In [15]:
def company(text):
    text = text.replace('+','')
    text = text.replace('-','')
    text = text.replace('Pty Ltd','')
    text = text.replace('Pty Limited','')
    return text

In [16]:
jobs['job_company']=jobs['job_company'].apply(lambda x: company(x))

In [17]:
jobs["job_company"] = jobs["job_company"].str.replace('Talent – Winner ‘Seek Large Recruitment Agency of the Year’ 3 consecutive years!','Talent')
jobs["job_company"] = jobs["job_company"].str.replace('Michael Page Information Technology','MichaelPage')
jobs["job_company"] = jobs["job_company"].str.replace('THE CAREER MEISTER','Career Meister')
jobs["job_company"] = jobs["job_company"].str.replace('Sirius Technology Sydney part of Sirius People','Sirius Technology')

In [18]:
jobs["job_company"].head(2)

0                   SEEK Limited
1    ecareer employment services
Name: job_company, dtype: object

### Job Location

I kept only six job location for analysis.

In [19]:
locations=["Sydney","Melbourne","Brisbane","Perth","ACT","Adelaide"]

In [20]:
jobs= jobs[jobs.job_location.isin(locations)]

In [21]:
jobs["job_location"].head(2)

0    Melbourne
1       Sydney
Name: job_location, dtype: object

### Job Title

>This column was the time consuming one because intially I had 786 unique job roles and I had to shrink them down to 15 for my project. Firstly, I created a simple function to fix **"\+" , "\-" , "\/"**. Then I wrote few functions and used regular expression to check the combination of various patterns occuring in the column job_title. For every job role I serached I create a new column in the dataset (**jobs**).

In [22]:
def title(text):
    text = text.replace('+',' ')
    text = text.replace('-',' ')
    text = text.replace('/',' ')
    text = text.replace('anayst','analyst')
    return text

In [23]:
jobs["job_title"] = jobs["job_title"].str.replace('BI','Business Intelligence')

In [24]:
jobs['job_title']=jobs['job_title'].apply(lambda x: title(x))

In [25]:
def data_sci(text):
    text = text.lower()
    pattern = re.compile('.*data\sscien(ce|tist).*')
    if re.match(pattern,text):
        return ("data scientist")
    else:
        return None

In [26]:
jobs['data_scientist'] = jobs['job_title'].map(lambda x: data_sci(x))

In [27]:
def data_anyst(text):
    text = text.lower()
    pattern = re.compile('.*[data\sanaly(st|tics)]?(analyst|analytics).*')
    if re.match(pattern,text):
        return ("data analyst")
    else:
        return None

In [28]:
jobs['data_analyst'] = jobs['job_title'].map(lambda x: data_anyst(x))

In [29]:
def data_engg(text):
    text = text.lower()
    pattern = re.compile('.*[data\sengineer]?(engineer|developer).*')
    if re.match(pattern,text):
        return ("data engineer")
    else:
        return None

In [30]:
jobs['data_engineer'] = jobs['job_title'].map(lambda x: data_engg(x))

In [31]:
def machine_l(text):
    text = text.lower()
    pattern = re.compile('.*machine\slearning.*')
    if re.match(pattern,text):
        return ("machine learning")
    else:
        return None

In [32]:
jobs['machine learning'] = jobs['job_title'].map(lambda x: machine_l(x))

In [33]:
def bus_int(text):
    text = text.lower()
    pattern = re.compile('.*business\sintelligence.*')
    if re.match(pattern,text):
        return ("business intelligence")
    else:
        return None

In [34]:
jobs['business_intelligence'] = jobs['job_title'].map(lambda x: bus_int(x))

In [35]:
def bus_anyst(text):
    text = text.lower()
    pattern = re.compile('.*business\sanaly(st|tics|sis).*')
    if re.match(pattern,text):
        return ("business analyst")
    else:
        return None

In [36]:
jobs['business_analyst'] = jobs['job_title'].map(lambda x: bus_anyst(x))

In [37]:
def research(text):
    text = text.lower()
    pattern = re.compile('.*[researc(h|er)]?(csiro).*')
    if re.match(pattern,text):
        return ("research fellow")
    else:
        return None

In [38]:
jobs['research'] = jobs['job_title'].map(lambda x: research(x))

In [39]:
def senior(text):
    text = text.lower()
    pattern = re.compile('.*(senior|lead|master|principal).*')
    if re.match(pattern,text):
        return ("senior")
    else:
        return None

In [40]:
jobs['senior'] = jobs['job_title'].map(lambda x: senior(x))

In [41]:
def junior(text):
    text = text.lower()
    pattern = re.compile('.*junior.*')
    if re.match(pattern,text):
        return ("junior")
    else:
        return None

In [42]:
jobs['junior'] = jobs['job_title'].map(lambda x: junior(x))

In [43]:
jobs.columns

Index(['job_area', 'job_classification', 'job_company', 'job_description',
       'job_location', 'job_salary', 'job_title', 'job_type', 'data_scientist',
       'data_analyst', 'data_engineer', 'machine learning',
       'business_intelligence', 'business_analyst', 'research', 'senior',
       'junior'],
      dtype='object')

#### Combining all the columns into one :- " job_name "

In [44]:
jobs['job_name'] = jobs[['data_scientist','data_analyst','data_engineer', 'machine learning',
                         'business_intelligence', 'business_analyst', 'research', 
                         'senior','junior']].apply(lambda x: ' '.join(filter(None,x)), axis=1)

In [45]:
jobs['job_name'].value_counts()

data analyst                                                 263
data engineer                                                208
                                                             162
data scientist                                                77
data engineer senior                                          67
data analyst senior                                           63
senior                                                        54
business intelligence                                         42
data engineer business intelligence                           42
data scientist senior                                         31
data analyst business analyst                                 24
business intelligence senior                                  24
data analyst business intelligence                            22
research fellow                                               12
data engineer machine learning                                12
data scientist data analy

#### Fixing the job-titles that have multiple responsiblities

In [46]:
jobs["job_name"]=jobs["job_name"].str.replace('data engineer business intelligence','business intelligence')

In [47]:
jobs["job_name"]=jobs["job_name"].str.replace('data analyst business analyst','data analyst')

In [48]:
jobs["job_name"]=jobs["job_name"].str.replace('data analyst business intelligence','business intelligence')

In [49]:
jobs["job_name"]=jobs["job_name"].str.replace('data engineer machine learning','data engineer')

In [50]:
jobs["job_name"]=jobs["job_name"].str.replace('data scientist data analyst','data scientist')

In [51]:
jobs["job_name"]=jobs["job_name"].str.replace('data analyst business intelligence senior','senior data analyst')

In [52]:
jobs["job_name"]=jobs["job_name"].str.replace('data engineer business intelligence senior','senior data engineer')

In [53]:
jobs["job_name"]=jobs["job_name"].str.replace('data analyst business intelligence business analyst','business intelligence')

In [54]:
jobs["job_name"]=jobs["job_name"].str.replace('data scientist data engineer','data scientist')

In [55]:
jobs["job_name"]=jobs["job_name"].str.replace('business intelligence business analyst','business intelligence')

In [56]:
jobs["job_name"]=jobs["job_name"].str.replace('data scientist machine learning','data scientist')

In [57]:
jobs["job_name"]=jobs["job_name"].str.replace('data analyst data engineer','data engineer')

In [58]:
jobs["job_name"]=jobs["job_name"].str.replace('data scientist business intelligence','data scientist')

In [59]:
jobs["job_name"]=jobs["job_name"].str.replace('data analyst machine learning','data analyst')

#### Fixing the Designation for junior roles.

In [60]:
jobs["job_name"]=jobs["job_name"].str.replace('data scientist junior','junior data scientist')

In [61]:
jobs["job_name"]=jobs["job_name"].str.replace('data engineer junior','junior data engineer')

In [62]:
jobs["job_name"]=jobs["job_name"].str.replace('data analyst junior','junior data analyst')

In [63]:
jobs["job_name"]=jobs["job_name"].str.replace('machine learning','data analyst')

In [64]:
jobs["job_name"]=jobs["job_name"].str.replace('data engineer senior junior','junior data engineer')

In [65]:
jobs["job_name"]=jobs["job_name"].str.replace('business intelligence junior','junior business intelligence')

#### Fixing the Designation for senior roles.

In [66]:
jobs["job_name"]=jobs["job_name"].str.replace('machine learning senior','senior data scientist')

In [67]:
jobs["job_name"]=jobs["job_name"].str.replace('research fellow senior','senior research fellow')

In [68]:
jobs["job_name"]=jobs["job_name"].str.replace('data analyst senior','senior data analyst')

In [69]:
jobs["job_name"]=jobs["job_name"].str.replace('data engineer senior','data scientist')

In [70]:
jobs["job_name"]=jobs["job_name"].str.replace('data scientist senior','senior data scientist')

In [71]:
jobs["job_name"]=jobs["job_name"].str.replace('business intelligence senior','senior business intelligence')

#### Keeping the 14 job_titles which I am interested in.

In [72]:
title =["data analyst","data engineer","data scientist","business intelligence","research fellow"
        "senior data analyst","senior data engineer","senior data scientist","senior business intelligence",
        "senior research fellow","junior data analyst","junior data engineer","junior data scientist",
        "junior business intelligence"]

In [73]:
jobs= jobs[jobs.job_name.isin(title)]

#### Looking at the shape of Dataset

In [74]:
jobs.shape

(898, 18)

#### Finally getting rid of the redundant columns 

In [75]:
jobs.drop(['data_scientist','data_analyst', 'data_engineer', 'machine learning','business_intelligence', 
           'business_analyst', 'research','senior', 'junior'],axis=1,inplace=True)

In [76]:
jobs.shape

(898, 9)

### Write it to a new csv file

In [79]:
jobs.to_csv("job_postings2.csv")